In [1]:
%pip install accelerate==0.17.0
%pip install bitsandbytes==0.37.0
%pip install flexgen==0.1.7
%pip install gradio==3.18.0
%pip install numpy
%pip install requests
%pip install rwkv==0.3.1
%pip install safetensors==0.3.0
%pip install sentencepiece
%pip install tqdm
%pip install datasets
%pip install git+https://github.com/zphang/transformers@llama_push

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may ne

In [2]:
import transformers
from datasets import load_dataset, load_from_disk
from transformers import LLaMAForCausalLM, LLaMATokenizer

In [3]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.13.1
True


In [ ]:
# t = torch.cuda.get_device_properties(0).total_memory
# r = torch.cuda.memory_reserved(0)
# a = torch.cuda.memory_allocated(0)
# f = r-a  # free inside reserved


In [8]:
torch.cuda.get_device_properties(1)

AssertionError: Invalid device id

In [5]:
load_in_8bit = False

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=load_in_8bit,
    device_map="auto",
)
model = model.to("cuda")
tokenizer = LLaMATokenizer.from_pretrained(
    "decapoda-research/llama-7b-hf", add_eos_token=True
)
tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 0; 15.78 GiB total capacity; 14.97 GiB already allocated; 86.94 MiB free; 14.97 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [6]:
dataset = "alpaca"  # "tiny" or "alpaca"
if dataset == "tiny":
    data = load_from_disk("tiny_alpaca_data/")
elif dataset == "alpaca":
    data = load_dataset("json", data_files="alpaca_data.json")
print(len(data["train"]))

Found cached dataset json (/home/ec2-user/.cache/huggingface/datasets/json/default-c6083352a35bf59d/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

52002


In [7]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 52002
    })
})

In [8]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Response:
{data_point["output"]}"""

In [9]:
# optimized for RTX 4090. for larger GPUs, increase some of these?
MICRO_BATCH_SIZE = 4  # this could actually be 5 but i like powers of 2
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 1  # we don't need 3 tbh
LEARNING_RATE = 3e-4  # the karpathy constant
CUTOFF_LEN = 256  # 256 accounts for about 96% of the data

data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=True,
        max_length=CUTOFF_LEN,
        padding="max_length",
    )
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=1,
        output_dir="pure-alpaca",
        save_total_limit=3,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train(resume_from_checkpoint=False)

model.save_pretrained("pure-alpaca")


Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

NotImplementedError: Cannot copy out of meta tensor; no data!

In [11]:
data["train"][1]

{'instruction': "Write a one-paragraph summary about Isaac Newton's mathematical discoveries.",
 'input': '',
 'output': "Sir Isaac Newton's mathematical discoveries revolutionized the way scientists think about the universe. He formulated the law of universal gravitation, which states that any two particles in the universe are attracted to one another by a force that is proportional to the product of their masses and inversely proportional to the square of their separation. He also developed calculus, which provides a mathematical description of how objects move. His work with optics and the refraction of light led him to develop the famous color wheel. His principles helped to establish the fields of kinematics and dynamics, and his laws are still used today to predict the trajectories of satellites and spacecraft.",
 'input_ids': [1,
  13866,
  338,
  385,
  15278,
  393,
  16612,
  263,
  3414,
  29889,
  14350,
  263,
  2933,
  393,
  7128,
  2486,
  1614,
  2167,
  278,
  2009,
 

In [13]:
PROMPT = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a poem about the following topic.

### Input:
Cars

### Response:"""

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
).to("cuda")
generation_output = model.generate(
    **inputs, return_dict_in_generate=True, output_scores=True, max_new_tokens=128
)


In [14]:
for s in generation_output.sequences:
    print(tokenizer.decode(s))

 Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a poem about the following topic.

### Input:
Cars

### Response:


In [12]:
generation_output.sequences

tensor([[    1, 13866,   338,   385, 15278,   393, 16612,   263,  3414, 29892,
          3300,  2859,   411,   385,  1881,   393,  8128,  4340,  3030, 29889,
         14350,   263,  2933,   393,  7128,  2486,  1614,  2167,   278,  2009,
         29889,    13,    13,  2277, 29937,  2799,  4080, 29901,    13,  6113,
           263, 26576,  1048,   278,  1494, 11261, 29889,    13,    13,  2277,
         29937, 10567, 29901,    13, 29907,  1503,    13,    13,  2277, 29937,
         13291, 29901,     2,     1]], device='cuda:0')

In [12]:
"" == True

False

In [14]:
if "sd":
    print("bla")

bla
